## Functions

Initial functions: 
* read_data: reads data and returns X and y
* transform_data: standarizes the features X to be in the same scale

Dimensionality reduction + Propensity scores:
* factor_analysis: reduces the dimension of X to a smaller number of factors
* interpret_factors: for each factor plots the loadings of the most important features composing it
* logistic_regression: computes propensity scores for each unit
* ps_balanced: repeatedly computes propensity scores with random balanced subsamples

Feature selection:
* feature_selection: selects subset of important features with sparisity and multiconinearity constraints

Distance matrix:
* distance_matrix: computes the matrix distance for all combinations of treatment and control pairs
* distance_matrix_hard_constraints: computes the matrix distance for all combinations of treatment and control pairs including penalty for hard constraint features
* ps_distance_matrix: computes the matrix distance for all combinations of treatment and control pairs considering only propensity scores

Matching algorithms:
* dist_matching: matching based on distance or propensity scores only
* dist_matching_PS_calipers: matching based on distance with propensity scores calipers
* dist_matching_PS_calipers_HC: matching based on distance with propensity scores calipers including hard constraints in the optimization for better time efficiency

Evaluation:
* asmd: computes the average standardized mean difference between treatment and control groups
* bias_matches: computes the bias among features for the matches
* ate: computes the average treatment effect of a matched data
* variance_matches: calculates the variance among matches
* variance_matches: plots the bias distribution for each feature
* bias_boxplot: plots the cumulative distribution of bias values for each feature

### Initial functions

In [ ]:
def read_data(path='../Data/Final/CLEAN_DATA_22_Imputed.csv', outcome='STAAR_Meets', treatment='SURVEY_CRIMSI'):
    """
    Read in data and split into covariates, treatment, and outcome.

    Parameters:
    ----------
    path : str
        Path to data.
    outcome : str
        Name of outcome variable.
    treatment : str
        Name of treatment variable.
    
    Returns:
    -------
    X : pd.DataFrame
        Covariates.
    T : pd.Series
        Treatment.
    y : pd.Series
        Outcome.
    """

    data = pd.read_csv(path)  #load data
    y = data[outcome] #define outcome variable
    T = data[treatment] #define treatment variable
    X = data.drop([outcome, treatment], axis=1) #define covariates
    return X, T, y

In [ ]:
def transform_data(X):
    """
    Transform data by standardizing numerical variables, binarizing binary variables, and one-hot encoding categorical variables.

    Parameters:
    ----------
    X : pd.DataFrame
        The feature matrix.
    
    Returns:
    -------
    X_trans: pd.DataFrame
        The standardized feature matrix.
    """
     
    bin_cols = [col for col in X.columns if set(X[col]) == {0, 1}] #define binary variables
    cat_cols = [col for col in X.columns if X[col].dtype == 'object' or X[col].nunique() < 8 and col not in bin_cols] #define categorical variables
    num_cols = [col for col in X.columns if col not in bin_cols + cat_cols] #define numerical variables

    # Define the column transformer to standardize numerical, binary, and categorical variables separately
    ct = ColumnTransformer([ ('num', StandardScaler(), num_cols), ('bin', Binarizer(), bin_cols), ('cat', OneHotEncoder(drop='first'), cat_cols)])

    # Fit the column transformer to the training data
    ct.fit(X)

    # Get the feature names after transformation (for interpretability)
    feature_names = num_cols + bin_cols + list(ct.named_transformers_['cat'].get_feature_names_out(cat_cols))

    # Transform the data and create a new DataFrame with the transformed features 
    X_trans = pd.DataFrame(ct.transform(X), columns=feature_names)

    return X_trans

### Dimensionality reduction + Propensity scores

In [ ]:
def factor_analysis(X, T, k=None):
    """
    Reduce the dimensionality of X using Factor Analysis to k number of factors.

    Parameters:
    ----------
    X : pd.DataFrame
        The feature matrix.
    T : pd.Series
        The treatment vector.
    k : int
        The number of factors to keep.

    Returns:
    -------
    X_factor : pd.DataFrame
        The factor matrix.
    fa : FactorAnalyzer
        The fitted FactorAnalyzer object.
    """

    # if k (# of factors) is not given, define k with the 1:10 rule to avoid overfitting which states that k should be 10% of the treatment size. Ex: if treatment size is 100, k = 10
    if k is None:
        k = int(np.floor(X[T==1].shape[0]/10)) # define k with the 1:10 rule
    
    fa = FactorAnalyzer(n_factors=k, rotation='varimax') # define FactorAnalyzer object
    fa.fit(X) # fit FactorAnalyzer object to features space X
    X_factor = fa.transform(X) # transform X to factor space
    X_factor = pd.DataFrame(X_factor).rename(columns=lambda x: 'Factor_' + str(x+1)) # create a new DataFrame with the factor space features
    return X_factor, fa # return the data with factors and the fitted FactorAnalyzer object for later use 

In [ ]:
def interpret_factors(X,T):
    """
    Interpret the factors from the factor analysis by plotting the loadings of the top 5 features for each factor.

    Parameters:
    ----------
    X : pd.DataFrame
        The feature matrix.
    T : pd.Series
        The treatment vector.
    """
    
    X_factor = factor_analysis(X, T)[1] # Run factor analysis 

    loadings = X_factor.loadings_ # Get the loadings of the factors 

    # Create a dataframe of the loadings 
    loadings_df=pd.DataFrame.from_records(loadings)
    loadings_df.columns = ['Factor_' + str(i+1) for i in range(loadings_df.shape[1])] # Rename the columns to Factor_1, Factor_2, etc.
    loadings_df.index = X.columns # Rename the rows to the feature names for interpretability

    # Plot for each factor the loadings of top 5 features contributing to the factor
    plt.rcParams['figure.figsize'] = [4, 4]
    for i in range(loadings_df.shape[1]):
        loadings_df.iloc[:,i].abs().sort_values(ascending=False)[:5].plot(kind='barh')
        plt.title('Factor ' + str(i+1))
        plt.ylabel('Loading')
        plt.show()

In [ ]:
def logistic_regression(X, T):
    """
    Compute propensity scores using logistic regression. Propensity scores are the probability of each row to belong to the treatment group.

    Parameters:
    ----------
    X : pd.DataFrame
        The feature matrix.
    T : pd.Series
        The treatment vector.
    
    Returns:
    -------
    propenity_scores : pd.Series
        The propensity scores.
    """

    # fit logistic regression model
    log_reg = LogisticRegression(max_iter=1000, random_state=0).fit(X, T)

    # predict probabilities for each row to belong to treatment group
    probs = log_reg.predict_proba(X)[:,1] 

    #convert probs to Series with index of X
    propensity_scores=pd.Series(probs, index=X.index)
    
    return propensity_scores

In [ ]:
def ps_balanced(X, T):
    """
    Repeatedly compute propensity scores with random subsamples of the control group to solve the class imbalance problem between treated and control.
    We repeat this process until all control units have a propensity score, and we average all the computed scores for the treated units to get the final propensity scores.

    Parameters:
    ----------
    X : pd.DataFrame
        The feature matrix.
    T : pd.Series
        The treatment vector.
    
    Returns:
    -------
    ps_df : pd.DataFrame
        The propensity scores for each subsample.
    propensity_scores_final : pd.Series
        The final propensity scores.
    """
    
    # Define control and treated groups
    control = X[T == 0]
    treated = X[T == 1]
    
    # Determine the number of subsamples to create. Ex: if control size is 100 and treated size is 10, we need 10 subsamples of control units 
    n_subsamples = int(np.ceil(control.shape[0] / treated.shape[0]))

    # Create a dataframe to store the propensity scores for each unit
    propensity_score_cols = [f'propensity_score_{i}' for i in range(n_subsamples)] # Create a list to store the propensity score columns
    ps_df = pd.DataFrame(columns=propensity_score_cols, index=X.index)
    
    # Randomly shuffle the control units to avoid bias in the subsamples
    shuffled_control = control.sample(frac=1, random_state=42)
    
    # Loop to compute propensity scores for each subsample
    for i in range(n_subsamples): # Loop over the number of subsamples
        # Calculate the start and end indices for the current subsample
        start_idx = i * treated.shape[0]  
        end_idx = min((i + 1) * treated.shape[0], control.shape[0]) # Make sure that the last subsample contains all the remaining control units
        
        control_sample = shuffled_control.iloc[start_idx:end_idx] # Get the control units for the current subsample
        
        sample = pd.concat([control_sample, treated], axis=0) # Combine the control sample and the treated group units to compute propensity scores
        
        sample_ps = logistic_regression(sample, T.loc[sample.index]) # Compute propensity scores for the sample
        
        # Append the propensity scores series to the dataframe
        ps_df.loc[sample_ps.index, f'propensity_score_{i}'] = sample_ps
    
    # Compute the final propensity score as the mean of the subsample propensity scores 
    ps_df['FINAL_PS'] = ps_df.mean(axis=1) 

    # Convert final propensity scores as a series with index of X
    propensity_scores_final=pd.Series(ps_df['FINAL_PS'], index=X.index)

    return ps_df, propensity_scores_final

### Feature selection

In [ ]:
def feature_selection(X, y, cv=5, method='lasso', num_selected=15, max_correlation=0.6, must_include=None):
    """
    Performs feature selection using Lasso, Ridge, or ElasticNet with optimization and adding multicollinearity and sparsity constraints.

    Parameters:
    ----------
    X : pd.DataFrame
        The feature matrix.
    y : pd.Series
        The outcome vector.
    cv : int
        Number of folds for cross-validation.
    method : str
        The method to use for feature selection. Must be 'lasso', 'ridge', or 'elastic'.
    num_selected : int
        The number of features to select.
    max_correlation : float
        The maximum correlation allowed between two features.
    must_include : list, optional
        The list of features that must be included in the matching model.
    
    Returns:
    -------
    selected_features : list
        The list of selected features names.
    coef_all : list
        The list of coefficients of all features.
    coef_selected : list
        The list of coefficients of selected features.

    """
    num_features = X.shape[1]

    # Define the model
    if method == 'lasso':
        model = LassoCV(cv=cv)
    elif method == 'ridge':
        model = RidgeCV(cv=cv)
    elif method == 'elastic':
        model = ElasticNetCV(cv=cv)
    else:
        raise ValueError('Method must be "lasso", "ridge", or "elastic"')

    # Fit the model to the data
    model.fit(X, y)

    # Get the optimal value of alpha and the corresponding coefficients
    alpha = model.alpha_
    coef = model.coef_

    # Create the optimization problem to include the sparsity and multicollinearity constraints
    prob = LpProblem("Feature_Selection", LpMaximize) # Define the optimization as a maximization problem

    # Create binary decision variables for each feature. If the feature is selected, the variable is 1, otherwise it is 0
    selected = LpVariable.dicts("selected", range(num_features), cat="Binary") 

    # Set the objective function: maximize the sum of absolute selected coefficients 
    prob += lpSum([selected[i] * abs(coef[i]) for i in range(num_features)])

    # Add constraints
    # Constraint 1: Avoid multicollinearity (correlation threshold)
    for i in range(len(X.columns)):
        for j in range(i + 1, len(X.columns)):
            if abs(pearsonr(X.iloc[:, i], X.iloc[:, j])[0]) > max_correlation: # If the correlation between the two features is greater than the threshold
                prob += selected[i] + selected[j] <= 1 # We can only select one of the two features to avoid multicollinearity

    # Constraint 2: Limit the number of selected features (sparsity constraint)
    prob += lpSum([selected[i] for i in range(len(X.columns))]) <= num_selected

    # Constraint 3: Must include certain features
    if must_include is not None:
        for feature in must_include:
            # Get the index of the feature and set x to 1 (selected)
            feature_index = X.columns.tolist().index(feature)
            prob += selected[feature_index] == 1


    # Solve the problem and don't show the output
    prob.solve(PULP_CBC_CMD(msg=0))

    selected_indices = [i for i in range(num_features) if selected[i].varValue == 1] # Get the selected feature indices
    selected_features = X.columns[selected_indices].tolist() # Get the selected feature names

    # Save coefficients of all features in a list
    coef_all = coef.tolist()

    # Save coefficients of selected features in a list
    coef_selected = [coef[i] for i in selected_indices]

    return selected_features, coef_all, coef_selected # Return the selected features names, the coefficients of all features, and the coefficients of selected features

### Distance matrix

In [ ]:
def distance_matrix(X, T, features=None, weights=None):
    """
    Calculate the weighted (absolute euclidean) distance matrix between treated and control units for the features specified.

    Parameters:
    -----------
    X : pandas.DataFrame
        The feature matrix.
    T : pd.series
        Series with treatment indicator. 1 if treatment, 0 if control.
    features : list, optional
        The list of features to be used for distance calculation (default: None). If None, all features are used.
    weights : list, optional
        The list of weights to be used for distance calculation (default: None). If None, features are not weighted.
    
    Returns:
    --------
    D : pandas.DataFrame
        The distance matrix where rows correspond to treated units and columns correspond to control units. 
        The row and column names are the indexes of the original data.

    """

    # Define treated and control units
    treated = X[T == 1]
    control = X[T == 0]

    # Define subset of features to be used for distance calculation
    if features is None: # If no features are specified, use all features
        features = X.columns
    
    # Define weights for each feature
    if weights is None: # If no weights are specified, use equal weights for all features
        weights = np.ones(len(features))

    # Select the values of the relevant features from treated and control units
    treated_features = treated[features].values
    control_features = control[features].values

    # Calculate the weighted distance matrix using vectorized operations
    weighted_diff = np.abs((treated_features[:, np.newaxis, :] - control_features) * weights).sum(axis=2) # Absolute weighted difference between treated and control units for each feature

    # Transform into a DataFrame and add row and column names as indexes of the original data
    D = pd.DataFrame(weighted_diff, index=treated.index, columns=control.index)

    return D

In [ ]:
def distance_matrix_hard_constraints(X, T, features=None, hard_constraint_features=None, weights=None, penalty=1e9):
    """
    Calculate the weighted (absolute euclidean) distance matrix between treated and control units for the features specified,
    adding penalty for hard constraints features that don't have the same value between treated and control units.

    Parameters:
    -----------
    X : pandas.DataFrame
        The feature matrix.
    T : pd.series
        Series with treatment indicator. 1 if treatment, 0 if control.
    features : list, optional
        The list of features to be used for distance calculation (default: None). If None, all features are used.
    weights : list, optional
        The list of weights to be used for distance calculation (default: None). If None, features are not weighted.
    hard_constraint_features : list, optional
        The list of features to be used for distance calculation with hard constraints (default: None). If None, no features have hard constraints.
    weights: list, optional
        The list of weights to be used for distance calculation (default: None). If None, features are not weighted.
    penalty : float, optional
        The penalty to be applied to the distance between treated and control units for features with hard constraints. If penalty is higher, we ensure features with hard constraints are closer.
    
    Returns:
    --------
    D : pandas.DataFrame
        The distance matrix where rows correspond to treated units and columns correspond to control units.
        The row and column names are the indexes of the original data.
    
    """
    
    # Define treated and control units
    treated = X[T == 1]
    control = X[T == 0]

    # Define subset of features to be used for distance calculation
    if features is None: # If no features are specified, use all features
        features = X.columns.tolist()
    else:
        features = features.copy() # Make a copy of the list of features to avoid modifying the original list

    # Check if hard constraint features are present in the list of features
    if hard_constraint_features is not None:
        missing_features = set(hard_constraint_features) - set(features) # Get the list of hard constraint features that are not in the list of features
        if missing_features: 
            features += missing_features # Add missing hard constraint features to the list of features
    
    # Define weights for each feature
    if weights is None: # If no weights are specified, use equal weights for all features
        weights = np.ones(len(features))
    elif len(weights) != len(features): # If the number of weights is not equal to the number of features, add weights neutral weights for the hard constraints features
        missing_weights = np.ones(len(features) - len(weights))
        weights = np.concatenate([weights, missing_weights])

    # Select the values of the relevant features from treated and control units
    treated_features = treated[features].values
    control_features = control[features].values

    # Calculate the abolsute distances for all features
    distances = np.abs((treated_features[:, np.newaxis, :] - control_features) * weights)

    # Apply penalty to features with hard constraints
    if hard_constraint_features is not None:
        hard_constraint_indices = [features.index(feature) for feature in hard_constraint_features] # Get the indices of the hard constraint features
        distances[:, :, hard_constraint_indices] *= penalty # Apply penalty to the distances of the hard constraint features
    
    # Sum the distances across features to get the final distance matrix
    D = np.sum(distances, axis=2)

    # Transform into a DataFrame and add row and column names as indexes of the original data
    D = pd.DataFrame(D, index=treated.index, columns=control.index)

    return D

In [ ]:
def ps_distance_matrix(T, PS):
    """
    Calculate the weighted (absolute euclidean) distance matrix between treated and control units for the propensity scores.

    Parameters:
    -----------
    T : pd.series
        Series with treatment indicator. 1 if treatment, 0 if control.
    PS : pd.series
        Series with propensity scores.
    
    Returns:
    --------
    D : pandas.DataFrame
        The distance matrix of propensity scores where rows correspond to treated units and columns correspond to control units. 
        The row and column names are the indexes of the original data.

    """

    # Define treated and control units propensity scores
    treated_ps = PS.loc[T[T == 1].index]
    control_ps = PS.loc[T[T == 0].index]
    
    # Calculate the absolute distance matrix using vectorized operations
    ps_diff = np.abs(treated_ps.values[:, np.newaxis] - control_ps.values)

    # Transform into a DataFrame and add row and column names as indexes of the original data
    D = pd.DataFrame(ps_diff, index=treated_ps.index, columns=control_ps.index)

    return D

### Matching algorithms

In [ ]:
def dist_matching(X, T, dist, with_replacement=True, optimal=False, num_matches=1):
    """
    This function takes in a dataframe with treatment and control units, a dataframe with distances between treatment and control units, and a matching type
    and returns a dataframe with matched units.

    Parameters:
    -----------
    X : pd.dataframe
        Feature matrix.
    T : pd.series
        Series with treatment indicator. 1 if treatment, 0 if control.
    dist : pd.dataframe
        Dataframe with distances between treatment (rows) and control units (columns).
    with_replacement : boolean
        Whether or not to match with replacement. 
    optimal : boolean
        Whether or not to use optimal matching.
    num_matches : int
        Number of control matches per treatment unit. If 1, one-to-one matching is performed. If >1, one-to-many matching is performed.
    
    Returns:
    --------
    match_full : dataframe
        Dataframe with matched units.
    """

    # Identify treatment and control units
    treatment = X[T == 1]
    control = X[T == 0]

    matches = [] # Create an empty list to store matches

    # -------- OPTION 1: WITH REPLACEMENT ----------
    if with_replacement:
        match_num = 1 # Initialize the match number
        for t_idx, t_row in treatment.iterrows(): # Iterate over the treatment units
            match_indices = dist.loc[t_idx][control.index].nsmallest(num_matches).index.tolist() # Get the indices of the k closest control units
            matches.extend([[t_idx, c_idx, match_num] for c_idx in match_indices]) # Add the matches to the matches list
            match_num += 1 # Increment the match number
    
    # -------- OPTION 2: WITHOUT REPLACEMENT ----------
    elif not with_replacement:
        # --------- OPTION 2A: GREEDY MATCHING ----------
        if not optimal:
            match_num = 1 # Initialize the match number
            matched_controls = set()  # Keep track of matched control units
            for t_idx, t_row in treatment.iterrows():  # Iterate over the treatment units
                available_controls = [c_idx for c_idx in control.index if c_idx not in matched_controls]
                control_indices = dist.loc[t_idx][available_controls].nsmallest(num_matches).index.tolist()
                matches.extend([[t_idx, c_idx, match_num] for c_idx in control_indices])
                matched_controls.update(control_indices)
                match_num += 1 # Increment the match number

        # --------  OPTION 2B: OPTIMAL MATCHING (OPTIMIZATION) ----------
        elif optimal:
            # Create optimization problem: minimization problem
            prob = pulp.LpProblem("Matching_Problem", pulp.LpMinimize)

            # Define decision variables: binary variable x_{ij} = 1 if treatment unit i is matched to control unit j, 0 otherwise
            x = pulp.LpVariable.dicts("x", [(t_idx, c_idx) for t_idx in treatment.index for c_idx in control.index], cat='Binary')

            # Objective function: minimize total distance
            prob += pulp.lpSum(dist.loc[t_idx][c_idx] * x[(t_idx, c_idx)] for t_idx in treatment.index for c_idx in control.index)

            # Constraint 1: each treatment unit is matched to at least the number of control specified
            for t_idx in treatment.index: 
                    prob += pulp.lpSum(x[(t_idx, c_idx)] for c_idx in control.index) >= num_matches  

            # Constraint 2: each control unit is matched to at most one treatment unit
            for c_idx in control.index:
                prob += pulp.lpSum(x[(t_idx, c_idx)] for t_idx in treatment.index) <= 1

            # Solve the optimization problem
            start_time = time.time()  # Record the start time
            prob.solve(pulp.GLPK_CMD(msg=0)) # Use GLPK solver and suppress output
            end_time = time.time()  # Record the end time
            execution_time = end_time - start_time # Calculate the execution time
            print("Execution Time:", execution_time, "seconds")

            # Retrieve the matches from the decision variables 
            match_num = 1  # Initialize the match number
            matched_treatment = set()  # Keep track of treatment units that have been matched
            for t_idx in treatment.index: # Iterate over the treatment units
                matched_control = []  # Keep track of control units matched with this specific treatment unit
                for c_idx in control.index: # Iterate over the control units
                    if pulp.value(x[(t_idx, c_idx)]) == 1: # If the decision variable is 1, the pair is a match
                        matched_control.append(c_idx) # Add the control unit index to the matched control list
                if matched_control: # If the matched control list is not empty
                    if t_idx not in matched_treatment: # If the treatment unit has not been matched yet, it is not in the set
                        for c_idx in matched_control: # Iterate over the matched control units to this specific treatment unit
                            matches.append([t_idx, c_idx, match_num]) # Add the matched pair to the matches list with the match number
                        matched_treatment.add(t_idx) # Add the treatment unit index to the matched treatment set to keep track of matched treatment units
                        match_num += 1 # Increment the match number
    
    # If the problem found matches then return the matches
    if matches:
        match_full = pd.DataFrame(columns=X.columns)  # Create an empty dataframe to store the matches including the features from X

        # Iterate over the matches and include the matched number column 
        for match in matches:
            match_treatment = treatment.loc[match[0]].to_frame().T # Get matched treatment unit features values from X
            match_control = control.loc[match[1]].to_frame().T # Get matched control unit features values from X
            match_treatment['match_num'] = match[2]  # Use the match number from the matches list
            match_control['match_num'] = match[2]  # Use the match number from the matches list
            match_full = pd.concat([match_full, match_treatment, match_control]) # Concatenate the matched treatment and control units to the match_full dataframe
                
        # Add a column for the treatment indicator from T
        match_full['treatment'] = T[match_full.index]

        #drop duplicated treatment rows that are added for when num_matches>1 (just for formatting purposes)
        match_full=match_full[~((match_full['treatment']==1) & (match_full['match_num'].duplicated()))]
        return match_full
    
    # If the problem did not find matches, then print a suggestion to relax the constraints
    elif not matches:
        print('** WARNING ** No matches found. You should try to relax your constraints in the optimization problem as the problem may be infeasible. Also, you can try to reduce the number of control matches as there might not be enough control units to match to.')
        return None

In [ ]:
def dist_matching_PS_calipers(X, T, PS, dist, with_replacement=True, optimal=False, num_matches=1, caliper=.02):
    """
    Matches treated and control units based on the distance matrix and propensity scores with calipers.

    Parameters:
    -----------
    X : pandas.DataFrame
        The feature matrix.
    T : pandas.Series
        Series with treatment indicator. 1 if treatment, 0 if control.
    PS : pandas.Series
        Series with propensity scores.
    dist : pandas.DataFrame
        The distance matrix where rows correspond to treated units and columns correspond to control units.
    with_replacement : bool, optional
        Whether to match with replacement (default: True).
    optimal : bool, optional
        Whether to perform optimal matching (default: False). Creates optimal pairing when with_replacement = False.
    num_matches : int, optional
        Number of control matches per treatment unit. If 1, one-to-one matching is performed. If >1, one-to-many matching is performed.
    caliper : float, optional
        The caliper to be applied to the distance between treated and control units for the propensity score. If caliper is smaller, we ensure units matched have similar propensity scores.
  
    Returns:
    --------
    match_full : pandas.DataFrame
        Dataframe with the matched units. The match_num column indicates the match number for each treated and control unit pair.

    """
    # Identify treatment and control units
    treatment = X[T == 1]
    control = X[T == 0]

    matches = [] # Create an empty list to store matches

    # -------- OPTION 1: WITH REPLACEMENT ----------
    if with_replacement:
        match_num = 1 # Initialize the match number
        for t_idx, t_row in treatment.iterrows(): # Iterate over the treatment units
            control_within_caliper = control[abs(PS.loc[t_idx] - PS.loc[control.index]) <= caliper] # Get the control units within the propensity score caliper
            match_indices = dist.loc[t_idx][control_within_caliper.index].nsmallest(num_matches).index.tolist() # Get the indices of the k closest control units within the caliper
            matches.extend([[t_idx, c_idx, match_num] for c_idx in match_indices]) # Add the matches to the matches list
            match_num += 1 # Increment the match number
        
    # -------- OPTION 2: WITHOUT REPLACEMENT ----------
    elif not with_replacement:
        # --------- OPTION 2A: GREEDY MATCHING ----------
        if not optimal:
            match_num = 1 # Initialize the match number
            matched_controls = set()  # Keep track of matched control units
            for t_idx, t_row in treatment.iterrows(): # Iterate over the treatment units
                available_controls = [c_idx for c_idx in control.index if c_idx not in matched_controls] # Safe the control units that have not been matched yet
                control_within_caliper = control[abs(PS.loc[t_idx] - PS.loc[available_controls.index]) <= caliper] # Get the control units within the propensity score caliper from the available controls
                control_indices = dist.loc[t_idx][control_within_caliper].nsmallest(num_matches).index.tolist() # Get the indices of the k closest control units within the caliper
                matches.extend([[t_idx, c_idx, match_num] for c_idx in control_indices]) # Add the matches to the matches list
                matched_controls.update(control_indices) # Add the matched control indices to the matched controls set to update the available controls
                match_num += 1 # Increment the match number
                
        # --------  OPTION 2B: OPTIMAL MATCHING (OPTIMIZATION) ----------
        if optimal:
            # Create optimization problem: minimization problem
            prob = pulp.LpProblem("Matching_Problem", pulp.LpMinimize)

            # Define decision variables: binary variable x_{ij} = 1 if treatment unit i is matched to control unit j, 0 otherwise
            x = pulp.LpVariable.dicts("x", [(t_idx, c_idx) for t_idx in treatment.index for c_idx in control.index], cat='Binary')

            # Objective function: minimize total distance
            prob += pulp.lpSum(dist.loc[t_idx][c_idx] * x[(t_idx, c_idx)] for t_idx in treatment.index for c_idx in control.index)

            # Constraint 1: each treatment unit is matched to at least the number of control specified within the caliper
            for t_idx in treatment.index:
                controls_within_caliper = control[abs(PS.loc[t_idx] - PS.loc[control.index]) <= caliper] # Get the control units within the propensity score caliper
                prob += pulp.lpSum(x[(t_idx, c_idx)] for c_idx in controls_within_caliper.index) >= num_matches 
            
            # Constraint 2: each control unit is matched to at most one treatment unit
            for c_idx in control.index:
                prob += pulp.lpSum(x[(t_idx, c_idx)] for t_idx in treatment.index) <= 1

            # Solve the optimization problem
            start_time = time.time()  # Record the start time
            prob.solve(pulp.GLPK_CMD(msg=0)) # Use GLPK solver and suppress output
            end_time = time.time()  # Record the end time
            execution_time = end_time - start_time # Calculate the execution time
            print("Execution Time:", execution_time, "seconds")

            # Retrieve the matches from the decision variables 
            match_num = 1  # Initialize the match number
            matched_treatment = set()  # Keep track of treatment units that have been matched
            for t_idx in treatment.index: # Iterate over the treatment units
                matched_control = []  # Keep track of control units matched with this specific treatment unit
                for c_idx in control.index: # Iterate over the control units
                    if pulp.value(x[(t_idx, c_idx)]) == 1: # If the decision variable is 1, the pair is a match
                        matched_control.append(c_idx) # Add the control unit index to the matched control list
                if matched_control: # If the matched control list is not empty
                    if t_idx not in matched_treatment: # If the treatment unit has not been matched yet, it is not in the set
                        for c_idx in matched_control: # Iterate over the matched control units to this specific treatment unit
                            matches.append([t_idx, c_idx, match_num]) # Add the matched pair to the matches list with the match number
                        matched_treatment.add(t_idx) # Add the treatment unit index to the matched treatment set to keep track of matched treatment units
                        match_num += 1 # Increment the match number
    
    # If the problem found matches then return the matches
    if matches:
        match_full = pd.DataFrame(columns=X.columns)  # Create an empty dataframe to store the matches including the features from X

        # Iterate over the matches and include the matched number column 
        for match in matches:
            match_treatment = treatment.loc[match[0]].to_frame().T # Get matched treatment unit features values from X
            match_control = control.loc[match[1]].to_frame().T # Get matched control unit features values from X
            match_treatment['match_num'] = match[2]  # Use the match number from the matches list
            match_control['match_num'] = match[2]  # Use the match number from the matches list
            match_full = pd.concat([match_full, match_treatment, match_control]) # Concatenate the matched treatment and control units to the match_full dataframe
                
        # Add a column for the treatment indicator from T
        match_full['treatment'] = T[match_full.index]

        #drop duplicated treatment rows that are added for when num_matches>1 (just for formatting purposes)
        match_full=match_full[~((match_full['treatment']==1) & (match_full['match_num'].duplicated()))]
        return match_full
    
    # If the problem did not find matches, then print a suggestion to relax the constraints
    elif not matches:
        print('** WARNING ** No matches found. You should try to relax your constraints in the optimization problem as the problem may be infeasible. Also, you can try to reduce the number of control matches as there might not be enough control units to match to.')
        return None

In [ ]:
def dist_matching_PS_calipers_HC(X, T, PS, dist, with_replacement=True, optimal=False, num_matches=1, caliper=.02,
                                 hard_constraints=None):
    """
    Matches treated and control units based on the distance matrix and propensity scores with calipers including hard constraints in the optimization.

    Parameters:
    -----------
    X : pandas.DataFrame
        The feature matrix.
    T : pandas.Series
        Series with treatment indicator. 1 if treatment, 0 if control.
    PS : pandas.Series
        Series with propensity scores.
    dist : pandas.DataFrame
        The distance matrix where rows correspond to treated units and columns correspond to control units.
    with_replacement : bool, optional
        Whether to match with replacement (default: True).
    optimal : bool, optional
        Whether to perform optimal matching (default: False). Creates optimal pairing when with_replacement = False.
    num_matches : int, optional
        Number of control matches per treatment unit. If 1, one-to-one matching is performed. If >1, one-to-many matching is performed.
    caliper : float, optional
        The caliper to be applied to the distance between treated and control units for the propensity score. If caliper is smaller, we ensure units matched have similar propensity scores.
    hard_constraints : list, optional
        The list of features to be used for distance calculation with hard constraints (default: None). If None, no features have hard constraints.
        
    Returns:
    --------
    match_full : pandas.DataFrame
        Dataframe with the matched units. The match_num column indicates the match number for each treated and control unit pair.

    """
    # Identify treatment and control units
    treatment = X[T == 1]
    control = X[T == 0]

    matches = [] # Create an empty list to store matches

    # -------- OPTION 1: WITH REPLACEMENT ----------
    if with_replacement:
        match_num = 1 # Initialize the match number
        for t_idx, t_row in treatment.iterrows(): # Iterate over the treatment units
            control_within_caliper = control[abs(PS.loc[t_idx] - PS.loc[control.index]) <= caliper] # Get the control units within the propensity score caliper
            match_indices = dist.loc[t_idx][control_within_caliper.index].nsmallest(num_matches).index.tolist() # Get the indices of the k closest control units within the caliper
            matches.extend([[t_idx, c_idx, match_num] for c_idx in match_indices]) # Add the matches to the matches list
            match_num += 1 # Increment the match number
        
    # -------- OPTION 2: WITHOUT REPLACEMENT ----------
    elif not with_replacement:
        # --------- OPTION 2A: GREEDY MATCHING ----------
        if not optimal:
            match_num = 1 # Initialize the match number
            matched_controls = set()  # Keep track of matched control units
            for t_idx, t_row in treatment.iterrows(): # Iterate over the treatment units
                available_controls = [c_idx for c_idx in control.index if c_idx not in matched_controls] # Safe the control units that have not been matched yet
                control_within_caliper = control[abs(PS.loc[t_idx] - PS.loc[available_controls.index]) <= caliper] # Get the control units within the propensity score caliper from the available controls
                control_indices = dist.loc[t_idx][control_within_caliper].nsmallest(num_matches).index.tolist() # Get the indices of the k closest control units within the caliper
                matches.extend([[t_idx, c_idx, match_num] for c_idx in control_indices]) # Add the matches to the matches list
                matched_controls.update(control_indices) # Add the matched control indices to the matched controls set to update the available controls
                match_num += 1 # Increment the match number
                
        # --------  OPTION 2B: OPTIMAL MATCHING (OPTIMIZATION) ----------
        if optimal:
             # Create optimization problem: minimization problem
            prob = pulp.LpProblem("Matching_Problem", pulp.LpMinimize)

            # Define decision variables: binary variable x_{ij} = 1 if treatment unit i is matched to control unit j, 0 otherwise
            x = pulp.LpVariable.dicts("x", [(t_idx, c_idx) for t_idx in treatment.index for c_idx in control.index], cat='Binary')

            # Objective function: minimize total distance
            prob += pulp.lpSum(dist.loc[t_idx][c_idx] * x[(t_idx, c_idx)] for t_idx in treatment.index for c_idx in control.index)

            # Constraint 1: each treatment unit is matched to at least the number of control specified
            for t_idx in treatment.index: 
                controls_within_caliper = control[abs(PS.loc[t_idx] - PS.loc[control.index]) <= caliper] # Get the control units within the propensity score caliper
                controls_within_caliper_hc = [c_idx for c_idx in controls_within_caliper.index if all(X.loc[t_idx, f] == X.loc[c_idx, f] for f in hard_constraints)] # filter the controls by those where the hard constraint features have the same value for treated and control in X
                prob += pulp.lpSum(x[(t_idx, c_idx)] for c_idx in controls_within_caliper_hc) >= num_matches

            # Constraint 2: each control unit is matched to at most one treatment unit
            for c_idx in control.index:
                prob += pulp.lpSum(x[(t_idx, c_idx)] for t_idx in treatment.index) <= 1

            # Solve the optimization problem
            start_time = time.time()  # Record the start time
            prob.solve(pulp.GLPK_CMD(msg=0)) # Use GLPK solver and suppress output
            end_time = time.time()  # Record the end time
            execution_time = end_time - start_time # Calculate the execution time
            print("Execution Time:", execution_time, "seconds")

            # Retrieve the matches from the decision variables 
            match_num = 1  # Initialize the match number
            matched_treatment = set()  # Keep track of treatment units that have been matched
            for t_idx in treatment.index: # Iterate over the treatment units
                matched_control = []  # Keep track of control units matched with this specific treatment unit
                for c_idx in control.index: # Iterate over the control units
                    if pulp.value(x[(t_idx, c_idx)]) == 1: # If the decision variable is 1, the pair is a match
                        matched_control.append(c_idx) # Add the control unit index to the matched control list
                if matched_control: # If the matched control list is not empty
                    if t_idx not in matched_treatment: # If the treatment unit has not been matched yet, it is not in the set
                        for c_idx in matched_control: # Iterate over the matched control units to this specific treatment unit
                            matches.append([t_idx, c_idx, match_num]) # Add the matched pair to the matches list with the match number
                        matched_treatment.add(t_idx) # Add the treatment unit index to the matched treatment set to keep track of matched treatment units
                        match_num += 1 # Increment the match number
    
    # If the problem found matches then return the matches
    if matches:
        match_full = pd.DataFrame(columns=X.columns)  # Create an empty dataframe to store the matches including the features from X

        # Iterate over the matches and include the matched number column 
        for match in matches:
            match_treatment = treatment.loc[match[0]].to_frame().T # Get matched treatment unit features values from X
            match_control = control.loc[match[1]].to_frame().T # Get matched control unit features values from X
            match_treatment['match_num'] = match[2]  # Use the match number from the matches list
            match_control['match_num'] = match[2]  # Use the match number from the matches list
            match_full = pd.concat([match_full, match_treatment, match_control]) # Concatenate the matched treatment and control units to the match_full dataframe
                
        # Add a column for the treatment indicator from T
        match_full['treatment'] = T[match_full.index]

        #drop duplicated treatment rows that are added for when num_matches>1 (just for formatting purposes)
        match_full=match_full[~((match_full['treatment']==1) & (match_full['match_num'].duplicated()))]
        return match_full
    
    # If the problem did not find matches, then print a suggestion to relax the constraints
    elif not matches:
        print('** WARNING ** No matches found. You should try to relax your constraints in the optimization problem as the problem may be infeasible. Also, you can try to reduce the number of control matches as there might not be enough control units to match to.')
        return None

### Evaluating matches

In [ ]:
def asmd(data, features, T='treatment', perspective=['individual', 'global']):
    '''
    Calculates the average standardized mean difference for each feature. ASMD = Abs(Mean Treated - Mean Control)
    
    Parameters:
    -----------
    data: pd.DataFrame
        The feature matrix with the treatment. 
    features: list
        The list of important features to be used for the balance calculation.
    T: str, optional
        The name of the treatment column (default: 'treatment').
    perspective: str
        The perspective from which to calculate the ASMD. Options are 'individual' or 'global'. Individual calculates the ASMD for each match, and global calculates the ASMD for the entire dataset.
    
    Returns:
    --------
    ASMD_by_feature: pd.Series
        A series with the average standardized mean difference for each feature. 
    '''
    # Identify treatment and control units    
    treatment = data[data[T] == 1] 
    control = data[data[T] == 0] 

    # ----------------- OPTION 1: GLOBAL ASMD ------------------
    if perspective=='global': # Calculate ASMD globally comparing treatment and control
        ASMD_by_feature = abs(treatment[features].mean() - control[features].mean()) # calculate ASMD for each feature
        ASMD_global=ASMD_by_feature.mean() # calculate global ASMD as the mean of the ASMDs for each feature
        print(f'ASMD: {ASMD_global}')
        return ASMD_by_feature
    # ----------------- OPTION 2: INDIVIDUAL ASMD ------------------
    elif perspective=='individual': # Calculate ASMD individually for each match 
        # Initialize an empty DataFrame to store the bias
        asmd_by_match = pd.DataFrame(index=data['match_num'].unique(), columns=features)

        # Calculate the bias for each feature within each match
        for match_num in data['match_num'].unique():
            match_treated = treatment[treatment['match_num'] == match_num]
            match_control = control[control['match_num'] == match_num]

            # Calculate the bias for each feature
            match_asmd = np.abs(match_treated[features].mean() - match_control[features].mean()) #this is like ASMD by match

            # Assign the bias values to the corresponding match and feature in the bias DataFrame
            asmd_by_match.loc[match_num] = match_asmd
        asmd_by_match.loc['ASMD_by_feature'] = asmd_by_match[features].mean() #calculate the mean ASMD for each feature across matches

        print(f'ASMD by matches: {asmd_by_match.loc["ASMD_by_feature"].mean()}')
        return asmd_by_match

In [ ]:
def bias_matches(M, T, matches, features=None):
    """
    Evaluate the matching by calculating the bias between the treated and control units within each match, differentiated by features.

    Parameters:
    -----------
    M: pd.DataFrame
        The feature matrix with the treatment and matches column.
    T: str
        The treatment variable column name.
    matches: str
        The column name with match identifier number for each pair or group of matched units.
    features: list, optional
        The list of features to calculate the bias (default: None). If None, all features are used.

    Returns:
    --------
    bias: pd.DataFrame
        A DataFrame with the calculated bias for each match and feature.
    """

    # Identify the treated and control units
    treated = M[M[T] == 1]
    control = M[M[T] == 0]

    # Initialize an empty DataFrame to store the bias for the features
    bias = pd.DataFrame(index=M[matches].unique(), columns=features)

    # Calculate the bias for each feature within each match
    for match_num in M[matches].unique(): # Iterate over the matches
        match_treated = treated[treated[matches] == match_num] 
        match_control = control[control[matches] == match_num]

        # Calculate the bias for each feature: the mean absolute difference between the treated and control units within each match
        match_bias = np.abs(match_treated[features].values - match_control[features].values).mean(axis=0)

        bias.loc[match_num] = match_bias # Assign the bias values to the corresponding match and feature in the bias DataFrame
    bias['Total_bias']=bias.sum(axis=1) # Calculate the total bias for each match
    bias.loc['Average_bias'] = bias[features].mean(axis=0) # Calculate the average bias for each feature

    print(f'Average bias by features: {bias.loc["Average_bias"].mean()}')
    print(f'Average total distance: {bias["Total_bias"].sum()}')
    return bias

In [ ]:
def ate(X, matched_df, outcome='STAAR_Meets'):
    """
    Calculate the average treatment effect (ATE) for a matched dataset.

    Parameters:
    -----------
    X : pandas.DataFrame
        Dataframe containing the features.
    matched_df : pandas.DataFrame
        Matched dataset with match numbers assigned to each unit.
    """
    # Identify the treated and control units
    mean_treated = X.iloc[matched_df[matched_df['treatment'] == 1].index][outcome].mean()
    mean_control = X.iloc[matched_df[matched_df['treatment'] == 0].index][outcome].mean()

    # Calculate the ATE
    ate = mean_treated - mean_control
    print(f'ATE: {ate}, Mean treated: {mean_treated}, Mean control: {mean_control}') 

In [ ]:
def variance_matches(M, T, matches, features):
    """
    Calculate variance for each match and feature in a matched dataset.

    Parameters:
    -----------
    M: pd.DataFrame
        The feature matrix with the treatment and matches column.
    T: str
        The treatment variable column name.
    matches: str
        The column name with match identifier number for each pair or group of matched units.
    features : list
        List of feature column names to evaluate variance and bias. 

    Returns:
    --------
    df_var: pd.DataFrame
        A DataFrame with the variance for each match and feature.
    """

    # Identify the treated and control units
    treated = M[M[T] == 1]
    control = M[M[T] == 0]

    # Get unique match numbers
    unique_matches = M['match_num'].unique()

    # Empty dataframes to store variance by match and feature
    df_var = pd.DataFrame(index=unique_matches, columns=features)

    for m in unique_matches:
        for f in features:
            df_var.loc[m, f] = M.loc[M['match_num'] == m, f].var()
            
    #for each column, calculate the mean of the variance across matches
    for i in df_var.columns:
        df_var.loc['Mean_var_features', i]=df_var[i].mean()
    
    print(f'Average variance across matches: {df_var.iloc[-1, :-1].mean()}')
    return df_var

In [ ]:
def bias_boxplot(matching, treatment, match_num, features):
"""
This function creates a boxplot for each feature bias you want to compare.

Parameters:
------------
matching: Dataframe
	The matched data.
treatment: str 
	The treatment variable column name.
matches: str
	The column name with match identifier number for each pair or group of matched units.
features: list, optional
	The list of features to calculate the bias (default: None). If None, all features are used.
"""

    bias_df = bias_matches(matching, treatment, match_num, features=features).iloc[:-1, :-1]
    bias_df = bias_df.apply(pd.to_numeric, errors='coerce')
    num_features = len(features)
    num_plots_per_line = min(num_features, 5)
    num_lines = -(-num_features // num_plots_per_line) # Ceiling division
    fig, axes = plt.subplots(nrows=num_lines, ncols=num_plots_per_line, figsize=(15, 4*num_lines))
    for i, line_axes in enumerate(axes):
        for j, ax in enumerate(line_axes):
        feature_idx = i * num_plots_per_line + j
        if feature_idx < num_features:
            feature = features[feature_idx]
            ax.boxplot(bias_df[feature])
            ax.set_title(feature, fontsize=8)
            ax.set_xticklabels([])
        else:
            ax.axis('off') # Hide excess subplots

    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()


In [ ]:
def cumulative_evaluation(matching, treatment, match_num, features):
"""
Plots the cumulative distribution of bias values for each feature.
Parameters:
------------
matching: Dataframe
	The matched data.
treatment: str 
	The treatment variable column name.
matches: str
	The column name with match identifier number for each pair or group of matched units.
features: list, optional
	The list of features to calculate the bias (default: None). If None, all features are used.
"""

    bias_df = bias_matches(matching, treatment, match_num, features=features).iloc[:-1, :-1]
    num_features = len(features)
    num_plots_per_line = min(num_features, 5)
    num_lines = -(-num_features // num_plots_per_line) # Ceiling division
    fig, axes = plt.subplots(nrows=num_lines, ncols=num_plots_per_line, figsize=(15, 4*num_lines))
    for i, line_axes in enumerate(axes):
        for j, ax in enumerate(line_axes):
        feature_idx = i * num_plots_per_line + j
        if feature_idx < num_features:
            feature = features[feature_idx]
            cum_dist = bias_df[feature].value_counts().sort_index().cumsum() / bias_df.shape[0]
            ax.plot(cum_dist.index, cum_dist.values)
            ax.set_title(feature, fontsize=8)
            ax.set_yticks(np.arange(0, 1.1, 0.25))
        else:
                ax.axis('off') # Hide excess subplots
    plt.tight_layout()
    plt.show()
